In [1]:
!pip install tensorflow tensorflow-gpu pandas matplotlib sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df['is_toxic'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)

In [7]:
from tensorflow.keras.layers import TextVectorization


In [8]:
X = df['comment_text']
y = df['is_toxic']

In [9]:
MAX_FEATURES = 200000

In [10]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')


In [11]:
vectorizer.adapt(X.values)

In [12]:
vectorized_text = vectorizer(X.values)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Convert TensorFlow tensors to NumPy arrays
vectorized_text_np = vectorized_text.numpy()
y_np = y.to_numpy()

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(vectorized_text_np, y_np, test_size=0.2, random_state=42)

# Convert the NumPy arrays back to TensorFlow tensors if needed
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Batch, shuffle, and prefetch the datasets
batch_size = 16

train_dataset = train_dataset.cache().shuffle(160000).batch(batch_size).prefetch(8)
test_dataset = test_dataset.cache().batch(batch_size).prefetch(8)

In [17]:
# Take a fraction of the training dataset
train_fraction = 0.1  # Use 10% of the training data
small_X_train = X_train[:int(len(X_train) * train_fraction)]
small_y_train = y_train[:int(len(y_train) * train_fraction)]

# Create a smaller training dataset
small_train_dataset = tf.data.Dataset.from_tensor_slices((small_X_train, small_y_train))
small_train_dataset = small_train_dataset.cache().shuffle(160000).batch(batch_size).prefetch(8)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

# Define the model
model = Sequential()

model.add(Embedding(MAX_FEATURES + 1, 32))  # Embedding layer
model.add(Bidirectional(LSTM(32, activation='tanh')))  # Bidirectional LSTM
model.add(Dense(128, activation='relu'))  # Dense layer 1
model.add(Dense(256, activation='relu'))  # Dense layer 2
model.add(Dense(128, activation='relu'))  # Dense layer 3
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])


In [20]:
history = model.fit(
    small_train_dataset,
    epochs=3,  # Adjust epochs as needed
    validation_data=test_dataset,
    verbose=1
)



Epoch 1/3
798/798 [==============================] - 1146s 1s/step - loss: 0.1921 - accuracy: 0.9366 - val_loss: 0.1502 - val_accuracy: 0.9503
Epoch 2/3
798/798 [==============================] - 1137s 1s/step - loss: 0.0796 - accuracy: 0.9747 - val_loss: 0.1500 - val_accuracy: 0.9506
Epoch 3/3
798/798 [==============================] - 1135s 1s/step - loss: 0.0356 - accuracy: 0.9885 - val_loss: 0.2186 - val_accuracy: 0.9500


In [22]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import numpy as np

y_pred = model.predict(test_dataset)

y_true = []
for _, labels in test_dataset:
    y_true.extend(labels.numpy())  # Convert to numpy array

y_pred_binary = np.where(y_pred > 0.5, 1, 0)


accuracy = accuracy_score(y_true, y_pred_binary)
f1 = f1_score(y_true, y_pred_binary)
roc_auc = roc_auc_score(y_true, y_pred)


# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")



1995/1995 [==============================] - 234s 117ms/step
Accuracy: 0.9500
F1 Score: 0.7222
ROC AUC: 0.9331


In [21]:
model.save("lstm_model.h5")  # Save model to a file
from google.colab import files

files.download("lstm_model.h5")  # Download HDF5 file

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>